In [27]:
import cvxpy as cp
import yfinance as yf
import pandas as pd
import numpy as np

#Parameters
ticker_symbol = "AAPL"
start = "2023-01-01"
end = "2024-01-01"

#Stock Data Retrieval
ticker = yf.Ticker(ticker_symbol)
historical_data = ticker.history(start=start, end=end)
closing_prices = historical_data['Close']
option_expirations = ticker.options

#Option Expiration
chosen_expiry = option_expirations[-1]  # Choose the last available expiration date
options_chain = ticker.option_chain(chosen_expiry)

#Strike And Option Prices
callables = options_chain.calls
putables = options_chain.puts
calls_data = callables[['strike', 'lastPrice']]
puts_data = putables[['strike', 'lastPrice']]

#Log Returns
log_returns = np.log(closing_prices / closing_prices.shift(1)).dropna()
daily_volatility = log_returns.std()
annualized_volatility = daily_volatility * np.sqrt(252)

S0 = closing_prices.iloc[-1] # Last closing price
K = calls_data['strike'].values # Strike prices of calls
T = (pd.to_datetime(chosen_expiry) - pd.to_datetime(end)).days / 365  # Time to expiration in years
r = 0.05  # Risk-free interest rate
sigma = annualized_volatility
N = 3 # Time steps

dt = T / N  # Time step size
u = np.exp(sigma * np.sqrt(dt))  # Up factor
d = 1 / u  # Down factor
p = (np.exp(r * dt) - d) / (u - d)

asset_tree = np.zeros((N + 1, N + 1))
for i in range(N + 1):
    for j in range(i + 1):
        asset_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

#Terminal Payoff At Maturity (Call Option)
option_payoff = np.zeros(N+ 1)
for j in range(N + 1):
    ST = asset_tree[j, N] # Stock price at maturity
    option_payoff[j] = max(0, ST - K[-1])  # Payoff for the last strike price

#Build The Full Return Matrix (Backward Induction)
option_tree = np.zeros_like(asset_tree)
for j in range(N + 1):
    option_tree[j, N] = option_payoff[j]  # Set terminal node values (payoffs)
for i in range(N - 1, -1, -1):
    for j in range(i + 1):
        option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])

# payoff_matrix = np.expand_dims(option_payoff, axis=0) # Shape (1, N + 1)

payoff_entries = []
for i in range(N + 1):
    for j in range(i + 1):
        payoff_entries.append(option_tree[j, i])
payoff_matrix = np.array(payoff_entries)
size = len(payoff_matrix)
payoff_matrix = payoff_matrix.reshape(size, 1)
payoff_matrix = np.tile(payoff_matrix, (1, size)) #Square matrix: same payoff to both players

#n_strategies = payoff_matrix.shape[1]  # Number of strategies (option holder)

x = cp.Variable(size)
y = cp.Variable(size)  # Variable for the other player (option writer)
v = cp.Variable()  # Value of the option (expected payoff)

holder_constraints = [
    cp.sum(x) == 1,  # Probabilities must sum to 1
    x >= 0,  # Probabilities must be non-negative
    cp.matmul(payoff_matrix, x) >= v  # Expected payoff >= v for any column
]

writer_constraints = [
    cp.sum(y) == 1,  # Probabilities must sum to 1
    y >= 0,  # Probabilities must be non-negative
    cp.matmul(payoff_matrix.T, y) <= v  # Expected payoff <= v for any row
]

constraints = holder_constraints + writer_constraints

objective = cp.Minimize(0)

problem = cp.Problem(objective, constraints)
problem.solve()

backtest_results = []

for expiry in ticker.options[-5:]:
    try:
        options_chain = ticker.option_chain(expiry)
        calls = options_chain.calls
        
        current_price = closing_prices.loc[:expiry].iloc[-1]
        nearest_strike = calls.iloc[(calls['strike'] - current_price).abs().argmin()]
        strike = nearest_strike['strike']
        market_price = nearest_strike['lastPrice']

        t_exp = (pd.to_datetime(expiry) - pd.to_datetime(end)).days / 365
        if t_exp <= 0:
            continue

        T = t_exp
        dt = T / N
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(r * dt) - d) / (u - d)

        tree = np.zeros((N + 1, N + 1))
        for i in range(N + 1):
            for j in range(i + 1):
                tree[j, i] = current_price * (u ** (i - j)) * (d ** j)

        payoff = np.zeros(N + 1)
        for j in range(N + 1):
            ST = tree[j, N]
            payoff[j] = max(0, ST - strike)

        opt_tree = np.zeros_like(tree)
        for j in range(N + 1):
            opt_tree[j, N] = payoff[j]
        for i in range(N - 1, -1, -1):
            for j in range(i + 1):
                opt_tree[j, i] = np.exp(-r * dt) * (p * opt_tree[j, i + 1] + (1 - p) * opt_tree[j + 1, i + 1])

        entries = []
        for i in range(N + 1):
            for j in range(i + 1):
                entries.append(opt_tree[j, i])
        matrix = np.array(entries)
        size = len(matrix)
        matrix = matrix.reshape(size, 1)
        matrix = np.tile(matrix, (1, size))

        x = cp.Variable(size)
        y = cp.Variable(size)
        v = cp.Variable()
        constraints = [
            cp.sum(x) == 1, x >= 0,
            cp.sum(y) == 1, y >= 0,
            matrix.T @ x >= v,
            matrix @ y <= v
        ]
        prob = cp.Problem(cp.Minimize(0), constraints)
        prob.solve()

        backtest_results.append({
            "expiry": expiry,
            "strike": round(float(strike), 2) if not isinstance(strike, float) else round(strike, 2),
            "market_price": round(float(market_price), 2),
            "nash_price": round(float(v.value), 2),
            "error": round(float(v.value - market_price), 2),
            "relative_error_%": round((float(v.value) - float(market_price)) / float(market_price) * 100, 2)
        })

    except Exception as e:
        print(f"Skipping expiry {expiry} due to error: {e}")

backtest_df = pd.DataFrame(backtest_results)
print(backtest_df)


       expiry  strike  market_price  nash_price   error  relative_error_%
0  2026-09-18   190.0         41.44      147.49  106.05            255.91
1  2026-12-18   190.0         44.68      156.22  111.54            249.63
2  2027-01-15   190.0         45.28      158.87  113.59            250.86
3  2027-06-17   190.0         49.26      173.16  123.90            251.52
4  2027-12-17   190.0         53.90      189.85  135.95            252.22
